In [17]:
# 1. Instala as bibliotecas necessárias
!pip install readdbc pandas openpyxl chardet

# 2. Faz o upload do arquivo .dbc
from google.colab import files
uploaded = files.upload()

# 3. Identifica nome do arquivo enviado
dbc_filename = list(uploaded.keys())[0]

# 4. Converte .dbc em .csv
import readdbc
csv_filename = dbc_filename.replace('.dbc', '.csv')
readdbc.dbc2csv(dbc_filename, csv_filename)
print(f"✅ Arquivo convertido e salvo como: {csv_filename}")

# 5. Detecta codificação
import chardet

with open(csv_filename, 'rb') as f:
    result = chardet.detect(f.read(10000))
    detected_encoding = result['encoding'] or 'latin1'  # fallback se não detectar
print(f"🔍 Codificação detectada (com fallback): {detected_encoding}")

# 6. Detecta o separador automaticamente ou tenta múltiplos
import pandas as pd
import csv

try:
    # Tenta detectar o separador com csv.Sniffer
    with open(csv_filename, 'r', encoding=detected_encoding) as f:
        sample = f.read(2048)
        dialect = csv.Sniffer().sniff(sample)
        detected_sep = dialect.delimiter
        print(f"🔍 Separador detectado automaticamente: {repr(detected_sep)}")

    # Lê com pandas
    df = pd.read_csv(csv_filename, encoding=detected_encoding, sep=detected_sep)

except (csv.Error, pd.errors.ParserError) as e:
    print("⚠️ Falha ao detectar automaticamente. Tentando separadores comuns...")

    # Tenta com lista de separadores
    possible_separators = [',', ';', '|', '\t', ':', ' ', '/', '\\', '.', '-', '_', '~', '^', '*', '+', '=', '#', '"', "'", '`']
    success = False

    for sep in possible_separators:
        try:
            df = pd.read_csv(csv_filename, encoding=detected_encoding, sep=sep)
            print(f"✅ CSV lido com sucesso usando separador: {repr(sep)}")
            detected_sep = sep
            success = True
            break
        except pd.errors.ParserError as e:
            print(f"❌ Erro com separador {repr(sep)}: {e}")
        except Exception as e:
            print(f"⚠️ Erro inesperado com separador {repr(sep)}: {e}")

    if not success:
        raise ValueError("❌ Não foi possível ler o CSV com nenhum dos separadores testados.")

# 7. Limpeza opcional dos dados
df.columns = df.columns.str.strip()  # Remove espaços dos nomes das colunas
df = df.loc[:, ~df.columns.str.match(r'^\s*$')]  # Remove colunas sem nome
df.dropna(how='all', inplace=True)  # Remove linhas totalmente vazias

# Mostra prévia
print("📊 Primeiras linhas do arquivo lido:")
display(df.head())

# 8. Salva como Excel
xlsx_filename = csv_filename.replace('.csv', '.xlsx')
df.to_excel(xlsx_filename, index=False, engine='openpyxl')
print(f"📁 Arquivo XLSX gerado com sucesso: {xlsx_filename}")

# 9. Faz o download dos arquivos
files.download(csv_filename)
files.download(xlsx_filename)



Saving DOFET11.DBC to DOFET11 (1).DBC
✅ Arquivo convertido e salvo como: DOFET11 (1).DBC
🔍 Codificação detectada (com fallback): latin1
⚠️ Falha ao detectar automaticamente. Tentando separadores comuns...
❌ Erro com separador ',': Error tokenizing data. C error: Expected 1 fields in line 19, saw 2

❌ Erro com separador ';': Error tokenizing data. C error: Expected 1 fields in line 22, saw 3

❌ Erro com separador '|': Error tokenizing data. C error: Expected 1 fields in line 18, saw 2

❌ Erro com separador '\t': Error tokenizing data. C error: Expected 3 fields in line 12, saw 4

❌ Erro com separador ':': Error tokenizing data. C error: Expected 1 fields in line 22, saw 2

❌ Erro com separador ' ': Error tokenizing data. C error: Expected 1 fields in line 25, saw 2

❌ Erro com separador '/': Error tokenizing data. C error: Expected 1 fields in line 20, saw 3

❌ Erro com separador '\\': Error tokenizing data. C error: Expected 1 fields in line 19, saw 2

❌ Erro com separador '.': Error t

ValueError: ❌ Não foi possível ler o CSV com nenhum dos separadores testados.